In [ ]:
# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [2]:
# Import libraries
import sys
#I am using a sklearn library from git. I need to add the path
#sys.path.append('/home/diego/devel/scikit-learn')
print sys.path

import numpy as np
import scipy
import pandas as pd
from sklearn.cross_validation import train_test_split

['', '/usr/lib/python27.zip', '/usr/lib/python2.7', '/usr/lib/python2.7/plat-linux2', '/usr/lib/python2.7/lib-tk', '/usr/lib/python2.7/lib-old', '/usr/lib/python2.7/lib-dynload', '/usr/lib/python2.7/site-packages', '/usr/lib/python2.7/site-packages/gtk-2.0', '/usr/lib/python2.7/site-packages/wx-3.0-gtk2', '/usr/lib/python2.7/site-packages/IPython/extensions', '/home/diego/.ipython', '/home/diego/devel/scikit-learn']


In [3]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [4]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students,n_features = np.shape(student_data)
n_features = n_features-1
target = student_data.as_matrix(["passed"])
n_passed = target.tolist().count(["yes"])
n_failed = target.tolist().count(["no"])
grad_rate = float(n_passed)/float(n_students)*100.0
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [5]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = pd.DataFrame(student_data[target_col])  # corresponding targets/labels

print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [6]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty
    # Check each column
    for col, col_data in X.iteritems(): 
        if col_data.dtype == object:
                col_data = col_data.replace(['yes', 'no'], [1, 0])
            # Note: This should change the data type for yes/no columns to int

            # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
                col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX


X_all = preprocess_features(X_all)
y_all = preprocess_features(y_all)

        
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))


Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [7]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
X_train, X_test, y_train, y_test = train_test_split(X_all.as_matrix(),y_all.as_matrix(),test_size=0.24,train_size=0.76) 
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
    <p style="color:green;">Classification trees are characterized by low complexity and scale very well with an high volume of data. However, they are also characterized by an high variance depending on the little variances on the dataset (a mistake on a split on the top of the tree will affect the overall performances of the model) and the predictions are not as accurate as many other models due to their structure.</p>
- Given what you know about the data so far, why did you choose this model to apply?
<p style="color:green;">As first choice, I decided to select a low complexity model and see how performs on the data. If the pattern is fairly easy to identify from the data, this model will perform well and it will probably by one of the most performant models due to its low complexity.</p>
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [8]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)
    return (end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score,make_scorer
from sklearn.grid_search import GridSearchCV

clf = DecisionTreeClassifier(  max_features=None, splitter='best', class_weight='balanced', max_leaf_nodes=2, 
                             criterion='entropy', max_depth=None, min_samples_leaf=1, min_samples_split=1,
                             presort=True  )

# Fit model to training data
train_classifier(clf, X_train, y_train.reshape(y_train.shape[0]))  # note: using entire training set here
print clf  # you can inspect the learned model by printing it
#print clf.best_params_

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=None, max_features=None, max_leaf_nodes=2,
            min_samples_leaf=1, min_samples_split=1,
            min_weight_fraction_leaf=0.0, presort=True, random_state=None,
            splitter='best')


/home/diego/devel/scikit-learn/sklearn/grid_search.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [9]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target, y_pred, pos_label=1, average='binary'), (end - start)
f1,predtime = predict_labels(clf, X_train, y_train.reshape(y_train.shape[0]))
print "F1 score for training set: {}".format(f1)


Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.825396825397


In [10]:
# Predict on test data
f1,predtime = predict_labels(clf, X_test, y_test.reshape(y_test.shape[0]))
print "F1 score for test set: {}".format(f1)

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.764705882353


In [25]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_time = train_classifier(clf, X_train, y_train)
    train_f1_score,prediction_time = predict_labels(clf, X_train, y_train)
    test_f1_score,predtime = predict_labels(clf, X_test, y_test)
    print "F1 score for training set: {}".format(train_f1_score)
    print "F1 score for test set: {}".format(test_f1_score)
    return [len(X_train), train_f1_score, test_f1_score, train_time, prediction_time]

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

# I create a table with all results. To do that, I run the classifier again on the first dataset.
classifiers = []

clf = DecisionTreeClassifier(  max_features=None, splitter='best', class_weight='balanced', max_leaf_nodes=2, 
                             criterion='entropy', max_depth=None, min_samples_leaf=1, min_samples_split=1,
                             presort=True  )
res = train_predict(clf,X_train,y_train.reshape(y_train.shape[0]),X_test,y_test.reshape(y_test.shape[0]))
res.insert(0,"Decision Tree")
classifiers.append(res)


#First subset: Data are unbalanced. This might imply poor performances with decision trees. 
#I create a balanced set and retrain the classifier

indexsort = np.argsort(y_train.reshape(300))
bal_X_train = X_train[indexsort]
bal_X_train = bal_X_train.reshape(X_train.shape)
bal_y_train = y_train[indexsort]
bal_y_train = bal_y_train.reshape(y_train.shape)

bal_n_failed = bal_y_train.tolist().count([0])

bal_X_train = bal_X_train[0:(bal_n_failed*2),:]
bal_y_train = bal_y_train[0:(bal_n_failed*2)]

#not sure if calling fit on the same object it is going to discard the old training data. I create a new object to be sure.
clf = DecisionTreeClassifier(  max_features=None, splitter='best', class_weight='balanced', max_leaf_nodes=2, 
                             criterion='entropy', max_depth=None, min_samples_leaf=1, min_samples_split=1,
                             presort=True  )

res = train_predict(clf,bal_X_train,bal_y_train.reshape(bal_y_train.shape[0]),X_test,y_test.reshape(y_test.shape[0]))
res.insert(0,"Decision Tree")
classifiers.append(res)

#training set of 100 examples

small_X_train = X_train[0:100]
small_y_train = y_train[0:100]
clf = DecisionTreeClassifier(  max_features=None, splitter='best', class_weight='balanced', max_leaf_nodes=2, 
                             criterion='entropy', max_depth=None, min_samples_leaf=1, min_samples_split=1,
                             presort=True  )

res = train_predict(clf,small_X_train,small_y_train.reshape(small_y_train.shape[0]),X_test,y_test.reshape(y_test.shape[0]))
res.insert(0,"Decision Tree")
classifiers.append(res)

------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.825396825397
F1 score for test set: 0.764705882353
------------------------------------------
Training set size: 196
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.75
F1 score for test set: 0.764705882353
------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Predicting labe

In [26]:
# TODO: Train and predict using two other models
#Second Model: SVC
from sklearn.svm import SVC
#from sklearn.model_selection import StratifiedShuffleSplit

param_grid = {'gamma':np.logspace(-9, 3, 13), 'C':np.logspace(-2, 10, 13)}
#cv = StratifiedShuffleSplit(n_iter=5, test_size=0.2, random_state=42)
clf = SVC(kernel='rbf',C=1.0, gamma=0.10000000000000001)

res = train_predict(clf,X_train,y_train.reshape(y_train.shape[0]),X_test,y_test.reshape(y_test.shape[0]))
res.insert(0,"SVC")
classifiers.append(res)

clf = SVC(kernel='rbf',C=1.0, gamma=0.10000000000000001)
res = train_predict(clf,bal_X_train,bal_y_train.reshape(bal_y_train.shape[0]),X_test,y_test.reshape(y_test.shape[0]))
res.insert(0,"SVC")
classifiers.append(res)

clf = SVC(kernel='rbf',C=1.0, gamma=0.10000000000000001)
res = train_predict(clf,small_X_train,small_y_train.reshape(small_y_train.shape[0]),X_test,y_test.reshape(y_test.shape[0]))
res.insert(0,"SVC")
classifiers.append(res)

print"###############################################################################################"

from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(15, weights='uniform')

res = train_predict(clf,X_train,y_train.reshape(y_train.shape[0]),X_test,y_test.reshape(y_test.shape[0]))
res.insert(0,"Nearest Neighbors")
classifiers.append(res)

clf = neighbors.KNeighborsClassifier(15, weights='uniform')

res = train_predict(clf,bal_X_train,bal_y_train.reshape(bal_y_train.shape[0]),X_test,y_test.reshape(y_test.shape[0]))
res.insert(0,"Nearest Neighbors")
classifiers.append(res)

clf = neighbors.KNeighborsClassifier(15, weights='uniform')

res = train_predict(clf,small_X_train,small_y_train.reshape(small_y_train.shape[0]),X_test,y_test.reshape(y_test.shape[0]))
res.insert(0,"Nearest Neighbors")
classifiers.append(res)

from tabulate import tabulate
print tabulate(classifiers, headers=['Classifier','train_size', 'F1_train', 'F1_test','train_time', 'pred_time'])

------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.011
Predicting labels using SVC...
Done!
Prediction time (secs): 0.008
Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
F1 score for training set: 0.973493975904
F1 score for test set: 0.807692307692
------------------------------------------
Training set size: 196
Training SVC...
Done!
Training time (secs): 0.005
Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.989898989899
F1 score for test set: 0.678899082569
------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.001
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for training set: 1.0
F1 score for test set: 0.79746835443
######################

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [35]:
# TODO: Fine-tune your model and report the best F1 score
param_grid = {'gamma':np.logspace(-9, 3, 20), 'C':np.logspace(-2, 10, 20)}
scorer = make_scorer(f1_score,greater_is_better=True,needs_proba=False,needs_threshold=False,average='binary')
clf = GridSearchCV(SVC(kernel='rbf'), param_grid=param_grid,scoring=scorer,cv=5,n_jobs=16)

res = train_predict(clf,X_train,y_train.reshape(y_train.shape[0]),X_test,y_test.reshape(y_test.shape[0]))
print tabulate([res], headers=['train_size', 'F1_train', 'F1_test','train_time', 'pred_time'])

print clf.best_params_

------------------------------------------
Training set size: 300
Training GridSearchCV...
Done!
Training time (secs): 95.549
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.007
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.838427947598
F1 score for test set: 0.813333333333
  train_size    F1_train    F1_test    train_time    pred_time
------------  ----------  ---------  ------------  -----------
         300    0.838428   0.813333       95.5492   0.00653911
{'C': 263.66508987303553, 'gamma': 0.00011288378916846884}
